In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.4.4 and Spark NLP 6.0.4
setup Colab for PySpark 3.4.4 and Spark NLP 6.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.8/718.8 kB 30.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.5 requires pyspark[connect]~=3.5.1, but you have pyspark 3.4.4 which is incompatible.


In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 6.0.4
Apache Spark version: 3.4.4


In [3]:
from sparknlp.base import DocumentAssembler, EmbeddingsFinisher, MultiDocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

In [4]:
from sparknlp.annotator import RoBertaEmbeddings

should use `roberta_base` by deafult, is using `roberta_base` ✅

In [5]:
documentAssembler = DocumentAssembler() \
     .setInputCol("text") \
     .setOutputCol("document")

tokenizer = Tokenizer() \
     .setInputCols(["document"]) \
     .setOutputCol("token")

embeddings = RoBertaEmbeddings.pretrained() \
     .setInputCols(["document", "token"]) \
     .setOutputCol("embeddings") \
     .setCaseSensitive(True)

embeddingsFinisher = EmbeddingsFinisher() \
     .setInputCols(["embeddings"]) \
     .setOutputCols("finished_embeddings") \
     .setOutputAsVector(True) \
     .setCleanAnnotations(False)

pipeline = Pipeline() \
     .setStages([
       documentAssembler,
       tokenizer,
       embeddings,
       embeddingsFinisher
     ])

data = spark.createDataFrame([
    ["This is a sentence."]
]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(finished_embeddings) as result").show(5, 80)

roberta_base download started this may take some time.
Approximate size to download 284.2 MB
[OK!]
+--------------------------------------------------------------------------------+
|                                                                          result|
+--------------------------------------------------------------------------------+
|[0.09378593415021896,0.13879825174808502,0.05456104129552841,-0.0982267111539...|
|[0.3181464672088623,0.19426730275154114,0.1143038421869278,-0.056538678705692...|
|[0.10648765414953232,-0.0962962955236435,-0.0707668587565422,-0.1927850395441...|
|[-0.05503883957862854,0.3008425533771515,-0.017401527613401413,-0.22713385522...|
|[0.21699899435043335,-0.4026758670806885,0.1304507702589035,0.260321527719497...|
+--------------------------------------------------------------------------------+



In [6]:
from sparknlp.annotator import RoBertaForQuestionAnswering

should use `roberta_base_qa_squad2` by deafult, is using `roberta_base_qa_squad2` ✅

In [7]:
documentAssembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier = RoBertaForQuestionAnswering.pretrained() \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")\
    .setCaseSensitive(True)

pipeline = Pipeline(stages=[
    documentAssembler,
    spanClassifier
])

data = spark.createDataFrame([
    ["What is my name?", "My name is Clara and I live in Berkeley."]
]).toDF("question", "context")

model = pipeline.fit(data)
result = model.transform(data)

result.select("answer.result").show(truncate=False)

roberta_base_qa_squad2 download started this may take some time.
Approximate size to download 441.7 MB
[OK!]
+-------+
|result |
+-------+
|[Clara]|
+-------+

